## 基于协同过滤的电影推荐

### User-Based CF 预测电影评分

In [2]:
import pandas as pd

In [35]:
ratings = pd.read_csv('../../data/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [37]:
# data_loc = ratings.loc[:3,:]
# print(data_loc)

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815


In [38]:
# data_ix = ratings.ix[:3,:3]
# print('data_ix:\n',data_ix)

data_ix:
    userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0


c:\users\struggle6\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [39]:
ratings = ratings.iloc[:,:3]
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


#### User-Movie的评分矩阵

In [18]:
# 透视表，将电影ID转换为列名称，转换成为一个User-Movie的评分矩阵
ratings_matrix = ratings.pivot_table(index=["userId"], columns=["movieId"],values="rating")

#计算用户之间相似度
user_similar = ratings_matrix.T.corr()

In [43]:
print(ratings_matrix)

movieId  1       2       3       4       ...  193583  193585  193587  193609
userId                                   ...                                
1           4.0     NaN     4.0     NaN  ...     NaN     NaN     NaN     NaN
2           NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
3           NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
4           NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
5           4.0     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
6           NaN     4.0     5.0     3.0  ...     NaN     NaN     NaN     NaN
7           4.5     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
8           NaN     4.0     NaN     NaN  ...     NaN     NaN     NaN     NaN
9           NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
10          NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
11          NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN

In [19]:
user_similar

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,0.079819,0.207983,0.268749,-2.916358e-01,-0.118773,0.469668,0.918559,-0.037987,-0.087643,NaN,0.878310,0.229081,0.232636,0.191851,0.037901,2.053708e-01,0.325180,0.419641,0.107119,-0.463039,-0.364372,0.249046,-0.166667,1.414214e-01,0.189437,-0.014257,-7.049479e-02,-0.142857,0.033366,0.277017,0.088523,0.032356,0.297044,0.432426,-0.419355,2.460849e-01,-0.225681,-0.354005,...,-0.096998,0.319158,0.054390,0.643796,0.422577,-0.500000,0.311132,NaN,-0.453563,0.116808,0.000000,0.250000,-0.037878,-0.076086,5.570311e-02,0.163573,-0.302195,-0.028161,-0.388075,0.311182,0.280875,-0.096225,0.492483,0.198460,0.509525,0.563226,0.427029,1.000000,0.219768,0.253649,0.091574,0.000000,-0.061503,-0.407556,-0.164871,6.637849e-02,0.174557,0.268070,-0.175412,-0.032086
2,NaN,1.000000,NaN,NaN,NaN,NaN,-0.991241,NaN,NaN,0.037796,NaN,NaN,NaN,NaN,-0.330174,-0.176471,0.000000,-2.646457e-01,NaN,NaN,-0.020535,-0.619292,NaN,-0.641689,0.108465,NaN,NaN,-0.166667,-3.100868e-01,NaN,NaN,NaN,1.000000,1.000000,NaN,-1.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,0.505370,NaN,NaN,NaN,NaN,NaN,NaN,-0.216506,-0.790569,0.044194,NaN,NaN,0.000000e+00,-0.197203,NaN,-1.000000,NaN,0.000000,NaN,-1.000000,-0.576557,NaN,NaN,0.494077,NaN,NaN,-0.016021,0.188982,-0.387347,NaN,-1.000000,NaN,NaN,5.833333e-01,NaN,-0.125000,NaN,0.623288
3,0.079819,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.944911,NaN,NaN,-3.545475e-01,-0.442498,NaN,-0.991241,NaN,NaN,NaN,NaN,NaN,NaN,0.204140,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-0.663403,NaN,NaN,0.779512,NaN,NaN,NaN,NaN,NaN,NaN,-0.500000,NaN,NaN,-0.471728,NaN,NaN,NaN,NaN,NaN,0.607764,-0.352130,NaN,0.197731,0.104257,NaN,NaN,0.433200,NaN,NaN,-7.913343e-01,-0.333333,-0.395092,NaN,0.569562
4,0.207983,NaN,NaN,1.000000,-0.336525,1.484982e-01,0.542861,0.117851,NaN,0.485794,0.564737,-0.344031,-0.558242,-0.139347,0.481605,-0.137553,-0.218928,-7.641214e-03,0.072000,0.620758,-0.059131,0.259412,-0.229480,0.151800,NaN,NaN,0.101143,0.125013,4.558423e-01,NaN,-0.198517,0.277774,0.166983,0.287883,0.000000,0.049130,-0.457496,3.792029e-01,0.071440,0.058747,...,-1.000000,-0.068616,-0.007547,0.707107,-0.311805,NaN,0.430248,NaN,-0.108578,-0.335836,0.688499,-0.613941,-0.793045,-0.127294,5.853661e-02,-0.170842,-0.259693,0.357217,-0.052342,0.388334,-0.024876,0.054554,0.239270,-0.082305,0.110432,-0.065938,0.202100,0.277350,0.080606,0.128722,-0.222113,0.396641,0.090090,-0.080296,0.400124,1.446033e-01,0.116518,-0.170501,-0.277350,-0.043786
5,0.268749,NaN,NaN,-0.336525,1.000000,4.316590e-02,0.158114,0.028347,NaN,-0.777714,0.268293,1.000000,NaN,-0.153323,-0.252352,-0.684653,0.310835,1.629953e-01,0.271827,0.040032,-0.339422,NaN,0.000000,0.193649,NaN,5.867387e-01,0.000000,0.201241,-6.813851e-01,0.500000,1.000000,0.325720,0.242925,0.000000,0.605530,NaN,0.297309,3.668257e-01,-0.326762,0.145549,...,NaN,0.654654,0.270031,0.054841,NaN,NaN,0.648105,NaN,-0.246183,0.059517,NaN,NaN,-0.612372,0.257629,NaN,-0.423659,0.704361,0.314098,0.484248,0.402967,NaN,-0.104103,0.327896,0.145599,NaN,0.406382,0.519907,NaN,0.398080,0.178820,0.000000,0.153303,0.234743,0.067791,-0.364156,2.443212e-01,0.231080,-0.020546,0.384111,0.040582
6,-0.291636,NaN,NaN,0.148498,0.043166,1.000000e+00,-0.126595,-0.200062,NaN,0.957427,0.483245,0.000000,NaN,0.276662,0.011639,-0.018377,0.024001,9.380339e-03,0.259189,0.213301,0.305718,0.896421,-0.405046,0.077758,NaN,-1.036641e-17,0.482119,-0.138378,7.145774e-01,1.000000,0.433831,-0.200959,0.024126,0.073892,-0.168365,0.281963,-0.111208,1.159176e-01,-0.360000,0.194852,...,NaN,0.000000,0.057356,0.542810,NaN,NaN,0.338568,NaN,0.405840,-0.022530,-0.500

In [24]:
ratings_matrix

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,4.0,5.0,3.0,5.0,4.0,4.0,3.0,NaN,3.0,4.0,NaN,3.0,NaN,4.0,4.0,4.0,NaN,2.0,NaN,2.0,5.0,NaN,4.0,3.0,4.0,3.0,NaN,NaN,NaN,3.0,4.0,4.0,5.0,NaN,NaN,NaN,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#1.找出uid 用户的相似用户
# similar_users = user_similar[1].dropna()  #按索引号切片
# print('similar_users0:\n',similar_users)
similar_users = user_similar[1].drop([1]).dropna()

# print('similar_users:\n',similar_users)
#1-2.相似用户筛选规则:正相关的用户
similar_users = similar_users.where(similar_users > 0).dropna()
# print('similar_users1:\n',similar_users)
#2.从用户1的近邻相似用户中筛选出对物品1有评分记录的近邻用户
ids = set(ratings_matrix[1].dropna().index)& set(similar_users.index)
finally_similar_users = similar_users.ix[list(ids)]



similar_users:
 userId
3      0.079819
4      0.207983
5      0.268749
6     -0.291636
7     -0.118773
8      0.469668
9      0.918559
10    -0.037987
11    -0.087643
13     0.878310
14     0.229081
15     0.232636
16     0.191851
17     0.037901
18     0.205371
19     0.325180
20     0.419641
21     0.107119
22    -0.463039
23    -0.364372
24     0.249046
25    -0.166667
26     0.141421
27     0.189437
28    -0.014257
29    -0.070495
30    -0.142857
31     0.033366
32     0.277017
33     0.088523
         ...   
581    0.000000
582    0.250000
583   -0.037878
584   -0.076086
585    0.055703
586    0.163573
587   -0.302195
588   -0.028161
589   -0.388075
590    0.311182
591    0.280875
592   -0.096225
593    0.492483
594    0.198460
595    0.509525
596    0.563226
597    0.427029
598    1.000000
599    0.219768
600    0.253649
601    0.091574
602    0.000000
603   -0.061503
604   -0.407556
605   -0.164871
606    0.066378
607    0.174557
608    0.268070
609   -0.175412
610   -0.032086
N

c:\users\struggle6\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [29]:

print('正相关用户 similar_users:\n',sim_users)
print('ids:\n',ids)
print('finally_similar_users:\n',finally_similar_users)

正相关用户 similar_users:
 userId
3      0.079819
4      0.207983
5      0.268749
6     -0.291636
7     -0.118773
8      0.469668
9      0.918559
10    -0.037987
11    -0.087643
13     0.878310
14     0.229081
15     0.232636
16     0.191851
17     0.037901
18     0.205371
19     0.325180
20     0.419641
21     0.107119
22    -0.463039
23    -0.364372
24     0.249046
25    -0.166667
26     0.141421
27     0.189437
28    -0.014257
29    -0.070495
30    -0.142857
31     0.033366
32     0.277017
33     0.088523
         ...   
581    0.000000
582    0.250000
583   -0.037878
584   -0.076086
585    0.055703
586    0.163573
587   -0.302195
588   -0.028161
589   -0.388075
590    0.311182
591    0.280875
592   -0.096225
593    0.492483
594    0.198460
595    0.509525
596    0.563226
597    0.427029
598    1.000000
599    0.219768
600    0.253649
601    0.091574
602    0.000000
603   -0.061503
604   -0.407556
605   -0.164871
606    0.066378
607    0.174557
608    0.268070
609   -0.175412
610   -0.03

In [41]:
#3.结合uid用户与近邻用户的相似度预测uid用户对iid物品的评分
numerator = 0 #分子
denominator = 0 # 分母
for sim_uid,similarity in finally_similar_users.iteritems():
    #近邻用户的评分数据
    sim_user_rated_movies = ratings_matrix.ix[sim_uid]
    #按住用户对iid物品的评分
    sim_user_rating_for_item = sim_user_rated_movies[1]
    #计算分子的值 
    numerator += similarity * sim_user_rating_for_item
    #计算分母的值
    denominator += similarity


c:\users\struggle6\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [42]:
# 计算预测的评分值
predict_rating = numerator/denominator
print("预测出用户1对电影1的评分：%0.2f" % ( predict_rating))

预测出用户1对电影1的评分：3.87


### Item-Based CF 预测电影评分

In [1]:
import pandas as pd
ratings = pd.read_csv('../../data/ml-latest-small/ratings.csv')
ratings = ratings.iloc[:,:3]

#### 转换成为一个User-Movie的评分矩阵

In [2]:
# 透视表，将电影ID转换为列名称，转换成为一个User-Movie的评分矩阵
ratings_matrix = ratings.pivot_table(index=["userId"], columns=["movieId"],values="rating")
#计算用户之间相似度
item_similar = ratings_matrix.corr()

In [3]:
print('ratings_matrix:\n',ratings_matrix)
print('item_similar:\n',item_similar)

ratings_matrix:
 movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     NaN     4.0     NaN     NaN     4.0     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
6           NaN     4.0     5.0     3.0     5.0     4.0     4.0     3.0   
7           4.5     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
8           NaN     4.0     NaN     NaN     NaN     NaN     NaN     NaN   
9           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
10          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
11          NaN     NaN     NaN     NaN     NaN     5.0     NaN     NaN   
12      

In [10]:
#1,找出iid 物品的相似物品
similar_items = item_similar[1].drop([1]).dropna()
#相似电影筛选,正相关的电影
similar_items = similar_items.where(similar_items > 0).dropna()
#2,从iid物品的近邻相似物品中筛选出uid用户评分过的物品
ids = set(ratings_matrix.ix[1].dropna().index) & set(similar_items.index)
# print(ids)
# ids1 = set(ratings_matrix.ix[1].dropna()) 
# print('----------\n',ids1)
finally_similar_items = similar_items.ix[list(ids)]


c:\users\struggle6\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\struggle6\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [11]:
print(finally_similar_items)

movieId
2048    0.254824
1025    0.236713
3       0.487109
1029    0.464971
6       0.106465
1030    0.295901
1031    0.451557
1032    0.241009
2054    0.232062
2058    0.298054
2571    0.158384
527     0.257732
1042    0.295109
2580    0.104383
2078    0.249286
543     0.167816
3617    0.343872
1060    0.253629
1573    0.221003
2596    0.537022
552     0.152544
553     0.495945
2090    0.237110
1580    0.255308
2094    0.139173
47      0.222006
2096    0.316594
1073    0.332187
50      0.250781
2099    0.782476
          ...   
423     0.149241
4006    0.831685
2985    0.368781
2987    0.233314
2991    0.339864
2997    0.036068
441     0.308539
954     0.468096
5060    0.249323
3527    0.226064
457     0.147549
2000    0.117128
2005    0.217781
3033    0.002690
3034    0.172324
1500    0.113119
2012    0.321453
480     0.245501
2528    0.444296
2018    0.196687
2529    0.279592
2028    0.202302
1517    0.095646
2542    0.338214
3052    0.043370
3053    0.189451
500     0.446261
3062  

In [12]:
# 3,结合iid 物品与其相似物品的相似度和uid用户对其相似物品的评分,预测uid对iid的评分
numerator = 0 #评分预测公式分子部分
denominator =0 #分母
for sim_iid,similarity in finally_similar_items.iteritems():
    #近邻物品的评分数据
    sim_item_rated_movies =ratings_matrix[sim_iid].dropna()
    #用户对相似物品的评分
    simItemRatingFromUser = sim_item_rated_movies[1]
    #计算分子的值
    numerator += similarity*simItemRatingFromUser
    #计算分母
    denominator += similarity



In [16]:
# 计算预测的评分值并返回
predict_rating = numerator/denominator
print(predict_rating)

4.3911501021913395
